# rendering a notebook with `pyscript`

* inject pyscript into the headers
* add ids to each output area
* make code cells py repls and target the output
* find dependencies and put them in the config

## `jinja2` template overrides for `nbconvert`

our __pyscript__ is going to be based off of the jupyterlab styling.

In [26]:
    %%file pyscript.j2
    {%- extends 'lab/index.html.j2' -%}

Overwriting pyscript.j2


In [27]:
    %%file -a pyscript.j2
    {%- block header -%}
    {{super()}}
    <link rel="stylesheet" href="https://pyscript.net/latest/pyscript.css" />
    <script defer src="https://pyscript.net/latest/pyscript.js"></script>
    {%- endblock header -%}

Appending to pyscript.j2


<var>requirejs</var> and <var>pyscript</var> do not play well together, and the block below prevents <var>requirejs</var> from loading.

In [28]:
    %%file -a pyscript.j2
    {%- block html_head_js -%}
    {%- endblock html_head_js -%}

Appending to pyscript.j2


from a notebook's source we can infer the packages it needs to run.
the packages are placed in the `<py-config>` tag

In [29]:
    %%file -a pyscript.j2
    {% block body_header %}   
    {{super()}}
    <py-config>
    packages = {{nb | get_imports_from_cells}}
    </py-config>
    {% endblock body_header %}

Appending to pyscript.j2


we replace cell inputs with a `<py-repl>` that does not autogenerate

In [30]:
    %%file -a pyscript.j2
    {% block input %}
    <py-repl output="out-{{cell.id}}">
    {{cell.source | escape | dedent}}
    </py-repl>
    {% endblock input %}
    

Appending to pyscript.j2


we use any existing outputs as the dead pixels.
the outputs are replaced the first pyscript executes in a nearby cell.

In [31]:
    %%file -a pyscript.j2
    {% block output %}
    <div id="out-{{cell.id}}">{{super()}}</div>
    {% endblock output %}
    
    {% block codecell %}
    {{super()}}
    {% if not cell.outputs %}
    <div id="out-{{cell.id}}"></div>
    {% endif %}
    {% endblock codecell %}
    

Appending to pyscript.j2


## `nbconvert` exporting machinery

In [49]:
    import depfinder, requests_cache, html, textwrap, bs4, requests
    from pathlib import Path; from functools import partial; from nbconvert.exporters import HTMLExporter, TemplateExporter
    requests_cache.install_cache()

### inferring dependencies

the `py-config` defines the environment. we use `depfinder` to do that.

In [50]:
    def get_imports_from_cells(nb):
        imports = set()
        for cell in nb.cells:
            imports.update(get_imports_from_cell(cell))
        if imports.intersection({"requests", "httpx", "urllib"}): # add more later
            imports.add("pyodide-http")
        return list(imports)
        
    def get_imports_from_cell(cell):
        if cell["cell_type"] == "code":
            try:
                yield from depfinder.inspection.get_imported_libs(textwrap.dedent("".join(cell["source"]))).required_modules
            except BaseException as e:
                pass

In [51]:
    exporter = partial(HTMLExporter, template_file="pyscript.j2", filters=dict(
        dedent=__import__('textwrap').dedent, get_imports_from_cells=get_imports_from_cells, escape=html.escape
    ))

In [54]:
    def get_pyscript(file):
        body, *_ = exporter().from_filename(file)
        return body

In [55]:
    def pyscript(file: Path, target: Path = None, write: bool=True):
        body = get_pyscript(file)
        if write:
            if not target:
                target = file.with_suffix(F"{file.suffix}.html")

            return target.write_text(body)

In [67]:
    if __name__ == "__main__":
        from IPython.display import *
        print(pyscript(Path("2022-12-19-integrating-typer.ipynb")))

587229
